# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
from math import atan2, degrees

In [2]:
mp_drawing = mp.solutions.drawing_utils # helpers for drawing
mp_pose = mp.solutions.pose

In [3]:
VIDEO_PATH = 'bicep-curl.mp4'
EXPORT_PATH = 'coords1.csv'
MODEL_PATH = 'bicep-curl.pkl'

# X. Make Some Detections

In [27]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    try:
        while cap.isOpened():
            ret, image = cap.read()

            if ret is False:
                print("Error Camera Not Found.")
            if image is None:
                print("Ignoring empty camera frame.")
                break

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            results = pose.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                             thickness= 2,
                                                             circle_radius= 4),
                                      mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                             thickness= 2,
                                                             circle_radius= 2)
                                     )
            
            # test code
            
            test = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            test.insert(0, 0)

            left_shoulder = (test[45], test[46])
            right_shoulder = (test[49], test[50])
            left_elbow = (test[53], test[54])
            right_elbow = (test[57], test[58])
            left_wrist = (test[61], test[62])
            right_wrist = (test[65], test[66])
            left_hip = (test[93], test[94])
            right_hip = (test[97], test[98])
            left_knee = (test[101], test[102])
            right_knee = (test[105], test[106])

            left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)

            left_lean_ang = angle(left_shoulder, left_hip, left_knee)
            right_lean_ang = angle(right_shoulder, right_hip, right_knee)

            left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
            right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)

            image = draw_text(image, f'Elbow {int(left_elbow_ang)}', (50, 100))
            # image = draw_text(image, f'Lean {int(left_lean_ang)}', (50, 150))
            image = draw_text(image, f'Shoulder {int(left_shoulder_ang)}', (50, 150))
            
            # end test code

            cv2.imshow('LUCIO', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    except Exception as e:
        cap.release()
        cv2.destroyAllWindows()
            
cap.release()
cv2.destroyAllWindows()

# 1. Save Video

In [5]:
cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter('bicep-curl.mp4', cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()

    try:
        cv2.imshow('Bicep-Curl', frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV

In [23]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [24]:
landmarks = ['class']
for val in range(0, 32 +1):
    landmarks += [f'x{val}', f'y{val}', f'z{val}', f'v{val}']
landmarks += ['left_elbow_ang', 'right_elbow_ang', 
              'left_lean_ang', 'right_lean_ang', 
              'left_shoulder_ang', 'right_shoulder_ang']
              # 'left_elbow_hip_dist', 'right_elbow_hip_dist']

In [32]:
with open(EXPORT_PATH, mode= 'w', newline= '') as f:
    csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
def angle_zibby(p1, p2, p3):
    # x1, y1 = p1
    # x2, y2 = p2
    # x3, y3 = p3
    # deg1 = (360 + degrees(atan2(x1 - x2, y1 - y2))) % 360
    # deg2 = (360 + degrees(atan2(x3 - x2, y3 - y2))) % 360
    # return deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)

    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3

    v21 = (x1 - x2, y1 - y2)
    v23 = (x3 - x2, y3 - y2)

    dot = v21[0] * v23[0] + v21[1] * v23[1]
    det = v21[0] * v23[1] - v21[1] * v23[0]

    theta = np.rad2deg(np.arctan2(det, dot))

    return theta

In [8]:
def angle(a, b, c):
    """ Calculate angle between two lines """
    if(a == (0,0) or b == (0,0) or c == (0,0)):
      return 0
    
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    # if angle > 180.0:
    #     angle = 360 - angle
        
    return angle

In [80]:
def dist_x(a, b):
    return (b[0] - a[0]) * 100

In [75]:
def dist_xy(a, b):
    diff_point1 = (a[0] - b[0]) ** 2
    diff_point2 = (a[1] - b[1]) ** 2
    return ((diff_point1 + diff_point2) ** 0.5) * 100

In [36]:
def draw_text(image, text, pos):
    """ Draw a intersection point in an image """
    cv2.putText(image,
                text,
                org= pos,
                fontFace= cv2.FONT_HERSHEY_PLAIN,
                fontScale= 1.5,
                color= (0, 255, 0),
                thickness= 3)

    return image

In [83]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [42]:
keypoints = [(res.x, res.y) for res in results.pose_landmarks.landmark]
keypoints

[0.5614598393440247,
 0.5916723012924194,
 -1.7221643924713135,
 0.9998307824134827,
 0.5812240839004517,
 0.4909902513027191,
 -1.6590086221694946,
 0.9996200203895569,
 0.6007997989654541,
 0.4845239222049713,
 -1.6589229106903076,
 0.9996513724327087,
 0.6203866004943848,
 0.4786299169063568,
 -1.6591726541519165,
 0.999580442905426,
 0.5134803056716919,
 0.5074714422225952,
 -1.6823654174804688,
 0.9996230602264404,
 0.48097705841064453,
 0.5133447051048279,
 -1.6813808679580688,
 0.9996510148048401,
 0.4519243836402893,
 0.5199608206748962,
 -1.6814537048339844,
 0.9996083974838257,
 0.6417275667190552,
 0.4857320189476013,
 -1.1103557348251343,
 0.9995972514152527,
 0.3889602720737457,
 0.5431966781616211,
 -1.1883807182312012,
 0.9997292160987854,
 0.5923542976379395,
 0.665318489074707,
 -1.4980854988098145,
 0.9998335242271423,
 0.5199892520904541,
 0.6942357420921326,
 -1.5216392278671265,
 0.9998555779457092,
 0.8218488693237305,
 0.841205894947052,
 -0.6982210874557495,
 0.

In [40]:
# left arm
angle_between(keypoints[11], keypoints[13], keypoints[15])

-162.28650607725368

In [41]:
# right arm
angle_between(keypoints[12], keypoints[14], keypoints[16])

159.75418028229322

In [33]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)
        
        # keypoint == keypoint * 4 + 1 (due to zero indentation and action)
        left_shoulder = (keypoints[45], keypoints[46])
        right_shoulder = (keypoints[49], keypoints[50])
        left_elbow = (keypoints[53], keypoints[54])
        right_elbow = (keypoints[57], keypoints[58])
        left_wrist = (keypoints[61], keypoints[62])
        right_wrist = (keypoints[65], keypoints[66])
        left_hip = (keypoints[93], keypoints[94])
        right_hip = (keypoints[97], keypoints[98])
        left_knee = (keypoints[101], keypoints[102])
        right_knee = (keypoints[105], keypoints[106])
        
        left_elbow_ang = angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_ang = angle(right_shoulder, right_elbow, right_wrist)
        
        left_lean_ang = angle(left_shoulder, left_hip, left_knee)
        right_lean_ang = angle(right_shoulder, right_hip, right_knee)
        
        left_shoulder_ang = angle(left_elbow, left_shoulder, left_hip)
        right_shoulder_ang = angle(right_elbow, right_shoulder, right_hip)
        
        # left_elbow_hip_dist = dist_xy(left_hip, left_elbow)
        # right_elbow_hip_dist = dist_xy(right_hip, right_elbow)
        
        keypoints.append(left_elbow_ang)
        keypoints.append(right_elbow_ang)
        keypoints.append(left_lean_ang)
        keypoints.append(right_lean_ang)
        keypoints.append(left_shoulder_ang)
        keypoints.append(right_shoulder_ang)
        
        # keypoints.append(left_elbow_hip_dist)
        # keypoints.append(right_elbow_hip_dist)
                
        with open(EXPORT_PATH, mode= 'a', newline= '') as f:
            csv_writer = csv.writer(f, delimiter= ',', quotechar= '"', quoting= csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
            
        return [f'Elbow(R): {int(right_elbow_ang)}', f'Lean(R): {int(right_lean_ang)}', f'Shoulder(R): {int(right_shoulder_ang)}']
            
    except Exception as e:
        print(f'Exception: {e}')
        pass

In [41]:
cap = cv2.VideoCapture(VIDEO_PATH)

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    right_elbow_ang = 0
    right_lean_ang = 0
    right_shoulder_ang = 0
    f = [f'Elbow(R): {right_elbow_ang}', f'Lean(R): {right_lean_ang}', f'Shoulder(R): {right_shoulder_ang}']
    
    try:
        while cap.isOpened():
            ret, image = cap.read()

            if ret is False:
                print("Error File Not Found.")
            if image is None:
                print("Ignoring empty camera frame.")
                break     

            # Recolor Feed
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            ## results = pose.process(image)
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                             thickness= 2,
                                                             circle_radius= 4),
                                      mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                             thickness= 2,
                                                             circle_radius= 2)
                                     )

            k = cv2.waitKey(1)
            if k == ord('a'):
                f = export_landmark(results, 'correct')
            elif k == ord('s'):
                f = export_landmark(results, 'error1')
            elif k ==ord('d'):
                f = export_landmark(results, 'error2')

            # print(f)
            image = draw_text(image, str(f[0]), (120, 500))
            image = draw_text(image, str(f[1]), (120, 550))
            image = draw_text(image, str(f[2]), (120, 600))


            cv2.imshow('Bicep Training', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
    except Exception as e:
        print(f'error {e}')
        cap.release()
        cv2.destroyAllWindows()

cap.release()
cv2.destroyAllWindows()

Error File Not Found.
Ignoring empty camera frame.


# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('coords.csv')

In [9]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.394216,0.823304,0.521883,0.999915,0.400480,0.832445,0.564747,0.999955,0.400534,...,-0.569941,0.905392,0.366447,0.239952,-0.529588,0.682359,0.366910,0.228794,-0.682824,0.893134
1,correct,0.378416,0.808986,0.500686,0.999824,0.388844,0.823843,0.544244,0.999917,0.389674,...,-0.469141,0.857716,0.349367,0.251711,-0.477883,0.606852,0.365400,0.247318,-0.577988,0.830602
2,correct,0.370498,0.800588,0.480671,0.999774,0.382954,0.818234,0.529139,0.999899,0.384757,...,-0.444559,0.837803,0.345535,0.257974,-0.451070,0.593900,0.364240,0.254299,-0.564721,0.809484
3,correct,0.391180,0.809407,0.553187,0.999752,0.394833,0.820088,0.587577,0.999887,0.395430,...,-0.364498,0.823966,0.346334,0.261834,-0.462412,0.603115,0.384745,0.262284,-0.452924,0.788278
4,correct,0.419454,0.826854,0.647878,0.999715,0.419590,0.834117,0.642996,0.999859,0.420833,...,-0.288368,0.806630,0.346707,0.259216,-0.518287,0.624147,0.386834,0.265655,-0.368683,0.762289


In [11]:
df.shape

(1409, 133)

In [12]:
df[df['class'] == 'correct']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,correct,0.394216,0.823304,0.521883,0.999915,0.400480,0.832445,0.564747,0.999955,0.400534,...,-0.569941,0.905392,0.366447,0.239952,-0.529588,0.682359,0.366910,0.228794,-0.682824,0.893134
1,correct,0.378416,0.808986,0.500686,0.999824,0.388844,0.823843,0.544244,0.999917,0.389674,...,-0.469141,0.857716,0.349367,0.251711,-0.477883,0.606852,0.365400,0.247318,-0.577988,0.830602
2,correct,0.370498,0.800588,0.480671,0.999774,0.382954,0.818234,0.529139,0.999899,0.384757,...,-0.444559,0.837803,0.345535,0.257974,-0.451070,0.593900,0.364240,0.254299,-0.564721,0.809484
3,correct,0.391180,0.809407,0.553187,0.999752,0.394833,0.820088,0.587577,0.999887,0.395430,...,-0.364498,0.823966,0.346334,0.261834,-0.462412,0.603115,0.384745,0.262284,-0.452924,0.788278
4,correct,0.419454,0.826854,0.647878,0.999715,0.419590,0.834117,0.642996,0.999859,0.420833,...,-0.288368,0.806630,0.346707,0.259216,-0.518287,0.624147,0.386834,0.265655,-0.368683,0.762289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,correct,0.330085,0.699986,-0.145416,0.997420,0.343068,0.721529,-0.112593,0.997978,0.345303,...,0.125256,0.379936,0.475425,-0.171171,0.599815,0.100177,0.403914,-0.206719,-0.051390,0.199322
535,correct,0.326041,0.694155,-0.125379,0.997486,0.337568,0.716694,-0.095789,0.998053,0.339866,...,0.075732,0.380614,0.492334,-0.196848,0.499532,0.099291,0.421936,-0.221657,-0.101301,0.200032
536,correct,0.328769,0.687269,-0.100638,0.997446,0.338549,0.709820,-0.072638,0.998046,0.340206,...,0.091619,0.375429,0.491100,-0.211724,0.507108,0.099538,0.420791,-0.236451,-0.098774,0.202052
537,correct,0.326496,0.680291,-0.051574,0.997505,0.335151,0.702815,-0.023659,0.998079,0.336878,...,0.097269,0.378330,0.494188,-0.209387,0.537105,0.099473,0.419598,-0.241676,-0.080947,0.205868


In [13]:
X = df.drop('class', axis= 1) 
y = df['class']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 7)

In [15]:
y_test

1083     error2
313     correct
734      error1
284     correct
671      error1
         ...   
619      error1
618      error1
816      error1
997      error2
779      error1
Name: class, Length: 423, dtype: object

## 3.2 Train Machine Learning Classification Model

In [16]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [17]:
pipelines = {
    'lr': make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [18]:
fit_models = {}

for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [19]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [20]:
fit_models['rc'].predict(X_test)

array(['error2', 'correct', 'error1', 'correct', 'error1', 'error1',
       'error1', 'correct', 'error2', 'error2', 'correct', 'correct',
       'correct', 'error1', 'error2', 'correct', 'error2', 'error2',
       'error1', 'correct', 'correct', 'correct', 'error2', 'correct',
       'error1', 'correct', 'error2', 'correct', 'correct', 'error2',
       'error1', 'error2', 'error2', 'correct', 'correct', 'correct',
       'error2', 'correct', 'correct', 'error1', 'correct', 'error1',
       'error1', 'correct', 'correct', 'error2', 'correct', 'error1',
       'error2', 'error2', 'correct', 'error2', 'error2', 'error2',
       'correct', 'error2', 'correct', 'error1', 'error2', 'error2',
       'error2', 'correct', 'error1', 'correct', 'error2', 'correct',
       'error2', 'correct', 'error1', 'correct', 'error1', 'error2',
       'error1', 'error1', 'correct', 'correct', 'correct', 'correct',
       'correct', 'error2', 'correct', 'correct', 'error2', 'correct',
       'error2', 'error

## 3.3 Evaluate and Serialize Model

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [22]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,
         accuracy_score(y_test.values, yhat),
         precision_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct"),
         recall_score(y_test.values, yhat, average= "micro", pos_label= "correct")
        )

lr 1.0 1.0 1.0 1.0
rc 0.9976359338061466 0.9976359338061466 0.9976359338061466 0.9976359338061466
rf 1.0 1.0 1.0 1.0
gb 0.9929078014184397 0.9929078014184397 0.9929078014184397 0.9929078014184397


c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
c:\Users\nasser\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1298: UserWarning: Note that pos_label (set to 'correct') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.war

In [23]:
yhat = fit_models['rf'].predict(X_test)

In [24]:
yhat[ :10]

array(['error2', 'correct', 'error1', 'correct', 'error1', 'error1',
       'error1', 'correct', 'error2', 'error2'], dtype=object)

In [25]:
with open(MODEL_PATH, 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [26]:
with open(MODEL_PATH, 'rb') as f:
    pickle.load(f)

In [ ]:
# X = pd.DataFrame([row], columns= landmarks[1:])

In [29]:
cap = cv2.VideoCapture(0)
counter = 0
current_stage = ''
state = ''

# Initiate holistic model
with mp_pose.Pose(min_detection_confidence= 0.5, min_tracking_confidence= 0.5) as pose:
    
    while cap.isOpened():
        ret, image = cap.read()
        # Recolor Feed
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color= (245, 117, 66),
                                                         thickness= 2,
                                                         circle_radius= 4),
                                  mp_drawing.DrawingSpec(color= (245, 66, 230),
                                                         thickness= 2,
                                                         circle_radius= 2)
                                 )
                
        try:
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row], columns= landmarks[1:])
            
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)
            correct = False; error1 = False; error2 = False

            if body_language_class == 'correct' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = True
                error1 = False
                error2 = False
            elif body_language_class == 'error1' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = True
                error2 = False
            elif body_language_class == 'error2' and body_language_prob[body_language_prob.argmax()] >= .7:
                correct = False
                error1 = False
                error2 = True
            
            if (error1):
                state = 'Elbow is out of place'
            elif (correct):
                state = 'Correct'
            elif(error2):
                state = 'Leaning'

            # elif current_stage = 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] >= .7:
            #     current_stage = 'up'
            #     counter += 1
            
            counter = 0
            
            
                # print(current_stage)

            # Get status box
            #cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            # Display Class
            cv2.putText(image, 
                        'CLASS',
                        (0,0),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            
            # cv2.putText(image, 
            #             body_language_class(' ')[0],
            #             (90,40),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,
            #             (255,255,255),
            #             2,
            #             cv2.LINE_AA
            #            )
            #Display Propability
            cv2.putText(image, 
                        'PROB',
                        (15,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(round(body_language_prob[np.argmax(body_language_prob)], 2)),
                        (10,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255,255,255),
                        2,
                        cv2.LINE_AA
                       )
            
            #Display Count
            cv2.putText(image, 
                        'COUNT',
                        (180,12),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (0,0,0),
                        1,
                        cv2.LINE_AA
                       )
            cv2.putText(image, 
                        str(state),
                        (175,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0,0,255),
                        2,
                        cv2.LINE_AA
                       )
            
            
        except Exception as e:
            pass


        cv2.imshow('Lucio', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()